In [ ]:
# Alice Clausing May 2023
# Gnu General Public License V3

In [1]:
import csv
from datetime import datetime
from os import path
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer

In [ ]:
# To create church directory:
# 1) Delete old export.csv file
# 2) Run "Church Directory Data as CSV" report in ChurchTrac for "Active Names" only, export in CSV format
# 3) Copy export.csv file to the same folder where the Jupyter notebook is 

In [2]:
# Generate church address list as PDF
# Text file is also created for debugging purposes, not intended for distribution.

EXPORT_PATH = "export.csv"
MAX_LINE_LEN = 70
NBSP10 = "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"

# file creation timestamp in float
c_time = path.getctime(EXPORT_PATH)
# convert creation timestamp into DateTime object
dt_c = datetime.fromtimestamp(c_time)

with open(EXPORT_PATH, newline='') as csvfile:
    # ChurchTrac doesn't provide column header for "Address 2" field, so we have to specify fieldnames
    export_reader = csv.DictReader(csvfile, fieldnames=('Formal', 'Primary', 'Address', 'Address2', 'Email', 'Comment'))
    f = open("church_directory.txt", "wt")
    
    # PDF setup
    doc = SimpleDocTemplate(
            "church_directory.pdf",
            pagesize=letter,
            rightMargin=50, leftMargin=50,
            topMargin=25, bottomMargin=25,
            )
    styles = getSampleStyleSheet()
    flowables = []   # list of items to be included in PDF
    bodyStyle = ParagraphStyle('Body', fontSize=12)
    para = Paragraph("Beaver UCC Directory" + NBSP10 + NBSP10 + NBSP10 + NBSP10 + NBSP10 + NBSP10 + NBSP10 + dt_c.strftime("%d %B %Y"), style=bodyStyle)
    flowables.append(para)    
    spacer = Spacer(1, 0.25*inch)
    flowables.append(spacer)
    
    first_row = True
    for row in export_reader:
        # print(row)   # debug - print dictionary contents
        # Skip column header row
        if first_row:
            first_row = False
        else:    
            family_name = row['Formal']

            # No column header for Address 2 when exported from ChurchTrac, but it's the 4th column in spreadsheet
            address2 = list(row.values())[3]
            address = row['Address'] + ', ' + address2 + '\n'
               
            f.write(family_name)
            f.write(address)
            if len(family_name) + len(address) < MAX_LINE_LEN:
                para = Paragraph(family_name + NBSP10 + address, style=bodyStyle)
                flowables.append(para)
            else:
                para = Paragraph(family_name, style=bodyStyle)
                flowables.append(para)
                para = Paragraph(address, style=bodyStyle)
                flowables.append(para)

            primary_phone = row['Primary']
            comment = row['Comment']
            if primary_phone in comment:
                phone_email = comment
            else:
                phone_email = primary_phone
                if len(comment) > 0:
                    phone_email = phone_email + NBSP10 + comment
            email = row['Email']
            if len(email) > 0:
                if len(phone_email) == 0:
                    phone_email = email
                elif len(phone_email) + 10 + len(email) > MAX_LINE_LEN:
                    f.write(phone_email + '\n')
                    para = Paragraph(phone_email, style=bodyStyle)
                    flowables.append(para) 
                    phone_email = email
                else:
                    phone_email = phone_email + NBSP10 + email
            if len(phone_email) > 0:        
                f.write(phone_email + '\n')  
                para = Paragraph(phone_email, style=bodyStyle)
                flowables.append(para) 
            f.write('\n')
            spacer = Spacer(1, 0.25*inch)
            flowables.append(spacer)
    f.close()  
    doc.build(flowables)
csvfile.close()            
        


In [11]:
csvfile.close()
f.close()

In [21]:
import csv
import random
import re

In [26]:
def obfuscate(str_in: str) -> str:
    """Replaces digit with random digit, alpha letter with random letter, leaving 1st character alone"""
    str_out = str_in[0]
    str_idx = 1
    str_len = len(str_in)
    while str_idx < str_len:
        char_in = str_in[str_idx]
        # print("obfuscate: str_idx=", str_idx, ", char_in=xxx", char_in, "xxx")   # debug
        if char_in.islower():
            str_out = str_out + random.choice("abcdefghijklmnopqrstuvwxyz")
        elif char_in.isupper(): 
            char_obf = random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
            str_out = str_out + char_obf
        elif char_in.isdigit():
            str_out = str_out + random.choice("0123456789")
        else:   # whitespace and punctuation characters do not change
            str_out = str_out + char_in
        str_idx = str_idx + 1 
    # print("obfuscate: str_in=xxx", str_in, "xxx, str_out=xxx", str_out, "xxx")  # debug  
    return str_out

In [43]:
def obfuscate_line(str_in: str, ignore_seq: list) -> str:
    """
    Obfuscates words and numbers in line.

    Input arguments:
    str_in: line to be obfuscated 
    ignore: sequence of words to ignore in line, i.e., words which will not be obfuscated. ignore_seq words should be in upper case, but the comparison is case-insensitive.
    """
    retstr = ""   # return string
    # See https://stackoverflow.com/questions/2136556/in-python-how-do-i-split-a-string-and-keep-the-separators
    # \W means "non-word" in regular expression
    # If capturing parentheses are used in pattern, then the text of all groups in the pattern are also returned as part of the resulting list.
    tokens_and_separators = re.split('(\W)', str_in)
    # if len(str_in) > 0:                                    # debug
    #    print("obfuscate_line: ", tokens_and_separators)   # debug
    for item in tokens_and_separators:
        if item.upper() in ignore_seq:
            retstr = retstr + item
        elif item.isalnum():
            retstr = retstr + obfuscate(item)
        else:
            retstr = retstr + item
    #if len(str_in) > 0:                                    # debug
    #    print("obfuscate_line returns ", retstr)           # debug
    return retstr

In [49]:
# Obfuscate the ChurchTrac CSV file, so it can be posted publicly as sample data file for this Jupyter notebook.
EXPORT_PATH = "export.csv"
FIELD_NAMES = ('Formal', 'Primary', 'Address', 'Address2', 'Email', 'Comment')

with open(EXPORT_PATH, newline='') as csvfile:
    # ChurchTrac doesn't provide column header for "Address 2" field, so we have to specify fieldnames
    export_reader = csv.DictReader(csvfile, FIELD_NAMES)
    f = open("church_directory.txt", "wt")
    
    with open('obfuscated.csv', 'w', newline='') as obffile:
        obfwriter = csv.writer(obffile, delimiter=',', dialect='excel',
                                quoting=csv.QUOTE_MINIMAL)

        first_row = True
        for row in export_reader:
            # print(row)   # debug - print ChurchTrac CSV row as dictionary
            if first_row:
                obfwriter.writerow(FIELD_NAMES)
                first_row = False
            else:    
                formal = obfuscate_line(row['Formal'], ('AND'))
                primary = obfuscate_line(row['Primary'], ())
                address = obfuscate_line(row['Address'], ("APT", "BOX", "CT", "DR", "DRIVE", "PO", "RD", "ST"))
                
                # No column header for Address 2 when exported from ChurchTrac, but it's the 4th column in spreadsheet
                address2 = obfuscate_line(list(row.values())[3], ("OH"))
                
                email = obfuscate_line(row['Email'], ("COM", "NET", "ORG"))
                comment = obfuscate_line(row['Comment'], ("CELL"))
                
                obfwriter.writerow([formal, primary, address, address2, email, comment])
                     



obfuscate_line:  ['Paul', ' ', 'and', ' ', 'Staci', ' ', 'Auer']
obfuscate_line returns  Plzi and Sngna Acqf
obfuscate_line:  ['3848', ' ', 'MESQUITE', ' ', 'DR']
obfuscate_line returns  3018 MDRTCLJG DR
obfuscate_line:  ['BEAVERCREEK', ' ', 'OH', ' ', '45440', '-', '3498']
obfuscate_line returns  BMGZJNVYKOT OH 40134-3298
obfuscate_line:  ['Ellen', ' ', 'Bagley']
obfuscate_line returns  Eptuj Bqkped
obfuscate_line:  ['457', ' ', 'BIG', ' ', 'STONE', ' ', 'DR']
obfuscate_line returns  437 BRE STCWV DR
obfuscate_line:  ['BEAVERCREEK', ' ', 'OH', ' ', '45434', '-', '5704']
obfuscate_line returns  BLPMOTYRIMH OH 45620-5235
obfuscate_line:  ['Bill', ' ', 'and', ' ', 'Karen', ' ', 'Barger']
obfuscate_line returns  Bspj and Kgswl Betdbt
obfuscate_line:  ['', '(', '205', ')', '', ' ', '531', '-', '2332']
obfuscate_line returns  (264) 565-2115
obfuscate_line:  ['2000', ' ', 'SUMAC', ' ', 'CT']
obfuscate_line returns  2188 SHMHH CT
obfuscate_line:  ['BEAVERCREEK', ' ', 'OH', ' ', '45431', '-', 